In [1]:
import os
import re
import json
import time
import random
import subprocess
from datetime import datetime

In [2]:
def get_speedtest_json():
    """Conducts a single speedtest and returns a json
    with the results.
    """
    
    # pass a subprocess command to cmd to get json
    try:
        out = subprocess.Popen(['speedtest.exe', '-f', 'json'],
                                stdout=subprocess.PIPE,
                                stderr=subprocess.STDOUT)
        stdout, stderr = out.communicate()
    except:
        pass
    
    # decode bytes and transform json into a dict
    try:
        _json = stdout.decode('utf-8')
        _dict = json.loads(_json)
        
        return(_dict)
    
    except (JSONDecodeError, UnboundLocalError, ValueError):
        _dict = {}
        
        # use a default dict
        _dict = {
                    "day": "2020-06-01",
                    "data": {
                        "time": [
                            "00:00:00"
                        ],
                        "download": [
                            0
                        ],
                        "upload": [
                            0
                        ]
                    },
                    "location": "None",
                    "computer": "None"
                }
        
        return(_dict)

In [5]:
_dict = get_speedtest_json()

In [6]:
def extract_speeds(_dict):
    """Given a dictionary of results from the speedtest,
    perform necessary transformations to return speeds.
    """
    
    # extract download and upload speeds in Mbps
    download = _dict['download']['bytes']/1e6
    upload = _dict['upload']['bytes']/1e6
    
    # return download an upload speeds as a tuple
    return(download, upload)

In [7]:
download, upload = extract_speeds(_dict)
download, upload

(62.936172, 5.732949)

In [8]:
def extract_ping(_dict):
    """Given a dictionary of results from the speedtest,
    perform necessary transformations to return ping stats.
    """
    
    jitter = _dict['ping']['jitter']
    latency = _dict['ping']['latency']

    return(jitter, latency)

In [9]:
jitter, latency = extract_ping(_dict)
jitter, latency

(0.524, 10.093)

In [4]:
def collect_info():
    """Perform 10 speed tests with up to 5 mins of wait time 
    between them and collect results.
    """
    
    # instantiate lists
    time_list = []
    jitter_list = []
    latency_list = []
    upload_list = []
    download_list = []
    
    for i in range(10):
        
        # wait random num secs for up to 5 mins
        secs = random.randint(10, 300)
        print('Loop ' + str(i+1) + ': waiting for ' \
              + str(secs) + ' secs...')
        time.sleep(secs)
        
        print('Performing speed test no.' + str(i+1))
        _dict = get_speedtest_json()
        
        # collect data
        download, upload = extract_speeds(_dict)
        jitter, latency = extract_ping(_dict)
        
        # add to lists
        print('Adding to lists.')
        jitter_list.append(jitter)
        latency_list.append(latency)
        download_list.append(download)
        upload_list.append(upload)
    
        # get dates and times
        now = time.time()
        dt_obj = datetime.fromtimestamp(now)
        
        # transform into readable format
        _day, _time = str(dt_obj).split(' ')
        _time = _time.split('.')[0]  

        # add to lists
        print('Adding to time list.')
        time_list.append(_time)
        
    # gather up lists into a dict
    _dict = {
             'day':_day,
             'data':{
                     'time':time_list,
                     'jitter':jitter_list,
                     'latency':latency_list,
                     'download':download_list,
                     'upload':upload_list
                    }
            }       
    
    return(_dict)

In [5]:
#if __name__=='__main__':
#    
#    # get user input 
#    # [TODO: sanitize, error check]
#    location = input("Enter room name: ") 
#    filename = input("Enter file name: ")
#    iters = int(input("Enter number of speed tests: "))
#    mins = int(input("Enter max waittime (mins) between tests: "))

In [6]:
def extract_filename():
    """Creates a filename by extracting the next name from
    filenames in data/.
    """
    
    # gets list of files and revers it using a natural key
    # this sorts correctly when going from 1 to 2+ digits
    filelist = os.listdir('data/')
    filelist.sort(key=lambda x: int(re.sub('\D', '', x))
                  , reverse=True)
    
    # get last digit and add one, create filename
    lastdigit = int(filelist[0].split('.')[0][4:6])+1
    filename = ''.join(['test', str(lastdigit), '.json'])
    
    return(filename)

In [7]:
filename = extract_filename()
filename

'test8.json'

In [11]:
def extract_machineinfo():
    """Use subprocess to pass a wmic command to extract 
    the machine name and info.
    """
    
    try:
        out = subprocess.Popen(['wmic', 'csproduct', 'get', 'name'],
                                stdout=subprocess.PIPE,
                                stderr=subprocess.STDOUT)
        stdout, stderr = out.communicate()
        
        # cleanup name
        names = stdout.decode('utf-8').split('\n')[1].split(' ')
        remove = ['', '\r\r', '\r', '\n']
        clean_names = [x for x in names if x not in remove]
        name = ' '.join([str(x) for x in clean_names]) 
    
        return(name)
    
    except (JSONDecodeError, UnboundLocalError, ValueError):
        name = 'No Name'
        
        return(name)

In [18]:
computer = extract_machineinfo()
computer = ' ' + computer + '  '
computer.strip()

'HP ProBook 450 G5'

In [16]:
_dir = 'data/'
if not os.path.exists(_dir):
    os.makedirs(_dir)

In [4]:
location = 'living room'
filename = 'debug01'
iters = 2
mins = 1

In [8]:
data = {'day': '2020-06-07',
 'data': {'time': ['11:40:50', '11:41:17'],
  'download': [42.576642, 93.752335],
  'upload': [4.205152, 3.093153]},
 'location': 'living room'}

In [4]:
# collect data, add location
data = collect_speeds(iters, mins)
data['location'] = location

Loop 1: waiting for 20 secs...
Performing speed test no.1
Adding to speed lists.
Adding to time list.
Loop 2: waiting for 8 secs...
Performing speed test no.2
Adding to speed lists.
Adding to time list.


In [7]:
data

{'day': '2020-06-07',
 'data': {'time': ['11:40:50', '11:41:17'],
  'download': [42.576642, 93.752335],
  'upload': [4.205152, 3.093153]},
 'location': 'living room'}

In [19]:
print('Writing out json.')
filepath = ''.join([_dir, str(filename), '.json'])
with open(filepath, 'w') as f:
    json.dump(data, f, indent=4)
           
print('Json file saved. Exiting now..')

Writing out json.
Json file saved. Exiting now..
